# NLP

En esta clase aprendimos los siguientes conceptos:

- Stop words
- Palabra raiz (Lemma)
- Tokenizado
- Vectorizado (BOW / TFIDF)

Ahora veremos como hacer esto en python.

Para NLP introduciremos algunas librerías nuevas, una de ellas es [Spacy](https://spacy.io/).

Otras librerías conocidas son:
- nltk
- gensim

Y una librería que es de lo mejor que hay en NLP actualmente: Hugging face.

Comenzaremos esta clase con Spacy. Ya viene pre-instalada en google colab por lo que no será necesario instalarla. Si luego trabajan en algún entorno en el que no este instalada, pueden seguir el tutorial de la página oficial.

Para usar spacy, debemos descargar un modelo del lenguaje que vayamos a trabajar. En este caso estaremos trabajando con textos en inglés (ya está descargado en colab), pero si en otro momento utilizan otro idioma, deben descargarlo desde https://spacy.io/models.

Ahora, importemos spacy y carguemos el modelo en inglés que utilizaremos en este notebook:




In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

### Carga de datos

El siguiente comando descargará un dataset de reviews de películas (en inglés) en su entorno de colab.

Luego de correr la siguiente celda, deberían ver en su entorno el directorio "acllmdb" que dentro contiene los datos.



In [ ]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar xvzf /content/aclImdb_v1.tar.gz

Se han truncado las últimas 5000 líneas del flujo de salida.
aclImdb/train/unsup/44983_0.txt
aclImdb/train/unsup/44982_0.txt
aclImdb/train/unsup/44981_0.txt
aclImdb/train/unsup/44980_0.txt
aclImdb/train/unsup/44979_0.txt
aclImdb/train/unsup/44978_0.txt
aclImdb/train/unsup/44977_0.txt
aclImdb/train/unsup/44976_0.txt
aclImdb/train/unsup/44975_0.txt
aclImdb/train/unsup/44974_0.txt
aclImdb/train/unsup/44973_0.txt
aclImdb/train/unsup/44972_0.txt
aclImdb/train/unsup/44971_0.txt
aclImdb/train/unsup/44970_0.txt
aclImdb/train/unsup/44969_0.txt
aclImdb/train/unsup/44968_0.txt
aclImdb/train/unsup/44967_0.txt
aclImdb/train/unsup/44966_0.txt
aclImdb/train/unsup/44965_0.txt
aclImdb/train/unsup/44964_0.txt
aclImdb/train/unsup/44963_0.txt
aclImdb/train/unsup/44962_0.txt
aclImdb/train/unsup/44961_0.txt
aclImdb/train/unsup/44960_0.txt
aclImdb/train/unsup/44959_0.txt
aclImdb/train/unsup/44958_0.txt
aclImdb/train/unsup/44957_0.txt
aclImdb/train/unsup/44956_0.txt
aclImdb/train/unsup/44955_0.txt
aclImdb/tra

Si navegan el directorio que tiene los datos, veran que hay un directorio train y otro test.

A su vez, dentro de cada uno de ellos podrán ver los directorios neg y pos. Ahi se encuentran los datos que utilizaremos hoy. 

En neg se encuentran reviews negativas, en pos review positivas.

La siguiente celda, lista los nombres de los archivos que hay en /content/aclImdb/test/pos.

Ven algo raro?


In [ ]:
!ls /content/aclImdb/test/pos/

0_10.txt      11875_8.txt   250_10.txt	 4376_10.txt  6252_8.txt   8128_8.txt
10000_7.txt   11876_10.txt  2501_7.txt	 4377_10.txt  6253_10.txt  8129_9.txt
10001_9.txt   11877_10.txt  2502_8.txt	 4378_7.txt   6254_7.txt   8130_10.txt
10002_8.txt   1187_7.txt    2503_10.txt  4379_10.txt  6255_10.txt  813_10.txt
10003_8.txt   11878_9.txt   2504_7.txt	 437_9.txt    6256_8.txt   8131_9.txt
10004_9.txt   11879_10.txt  2505_10.txt  4380_9.txt   6257_8.txt   8132_7.txt
10005_8.txt   11880_10.txt  2506_7.txt	 4381_9.txt   6258_8.txt   8133_8.txt
10006_7.txt   1188_10.txt   2507_10.txt  4382_7.txt   6259_7.txt   8134_7.txt
10007_10.txt  11881_10.txt  2508_10.txt  4383_10.txt  6260_9.txt   8135_8.txt
10008_8.txt   11882_10.txt  2509_7.txt	 4384_10.txt  626_10.txt   8136_7.txt
10009_10.txt  11883_10.txt  2510_7.txt	 4385_8.txt   6261_8.txt   8137_8.txt
1000_9.txt    11884_10.txt  25_10.txt	 4386_9.txt   6262_8.txt   8138_9.txt
10010_9.txt   11885_9.txt   2511_10.txt  4387_9.txt   6263_10.txt  8139_

Podemos ver que cada review está en un archivo .txt distinto. ¿Cómo podemos leer este tipo de archivos en pandas? Hasta ahora veníamos levantando únicamente CSVs.

En python, se pueden abrir achivos con el comando:



```
with open("ruta_al_archivo", modo_de_lectura) as f:
  # Acá ya tenemos acceso al archivo con el nombre f
```

donde modo_de_lectura puede ser:

- r: read
- w: write
- a: append

Más detalles: https://www.w3schools.com/python/ref_func_open.asp

Además, si importamos el paquete 

```
import os
```

podremos utilizar una función para listar el nomrbe de todos los archivos que se encuentran en un directorio:

```
os.listdir("directorio")
```

Entonces, lo que vamos a hacer es abrir el directorio donde se encuentran los archivos y leerlos uno por uno. A todo esto lo guardaremos luego en un dataframe de pandas.

Ejemplo de listado de reviews negativas con os.listdir:

In [ ]:
import os

os.listdir("/content/aclImdb/train/neg/")

['6243_1.txt',
 '5471_3.txt',
 '1534_1.txt',
 '1744_3.txt',
 '4736_2.txt',
 '9463_1.txt',
 '9827_1.txt',
 '7416_3.txt',
 '4163_3.txt',
 '8073_1.txt',
 '5241_3.txt',
 '9383_3.txt',
 '11505_3.txt',
 '10453_1.txt',
 '8710_1.txt',
 '4210_2.txt',
 '453_3.txt',
 '8662_1.txt',
 '3875_1.txt',
 '5880_1.txt',
 '4922_4.txt',
 '3670_3.txt',
 '4505_2.txt',
 '6450_1.txt',
 '7768_1.txt',
 '10811_1.txt',
 '7126_1.txt',
 '8633_4.txt',
 '10549_2.txt',
 '11654_4.txt',
 '6241_3.txt',
 '7564_1.txt',
 '9166_1.txt',
 '249_3.txt',
 '10063_1.txt',
 '10479_2.txt',
 '1037_1.txt',
 '7339_3.txt',
 '2167_2.txt',
 '7573_2.txt',
 '12259_1.txt',
 '11351_1.txt',
 '536_4.txt',
 '11166_1.txt',
 '8758_4.txt',
 '3501_1.txt',
 '9460_1.txt',
 '8498_3.txt',
 '6887_3.txt',
 '4872_4.txt',
 '5099_1.txt',
 '2827_3.txt',
 '4156_1.txt',
 '9707_3.txt',
 '2736_4.txt',
 '12139_2.txt',
 '8390_3.txt',
 '2794_3.txt',
 '58_3.txt',
 '5486_3.txt',
 '3180_2.txt',
 '11481_1.txt',
 '2595_3.txt',
 '4089_1.txt',
 '506_2.txt',
 '1904_1.txt',
 '87

Ahora, con un bucle for leemos todas las reviews negativas y las guardamos en una lista:

In [ ]:
import numpy as np
import pandas as pd

dir_neg_train = "/content/aclImdb/train/neg/"
neg_reviews = []

for f in os.listdir(dir_neg_train):
  with open(f"{dir_neg_train}/{f}") as neg:
    neg_reviews.append(neg.read())

Imprimimos las primeras 3 para corroborar que nuestro código funcione bien:

In [ ]:
neg_reviews[:3]

['Don\'t drink the cool-aid.<br /><br />This is an opinion piece disguised as a documentary. And to title it as a "truth" is just plain crap. The debate over global warming is far from over, and will only be over when the eco-zombies start acknowledging the mountain of evidence contrary to their beloved theory. Just Google "Global Warming" and "Hoax" or "Junk Science" and you will find a river of information refuting nearly every link in the chain of logic that Gore sites. The reason it is so important for people to educate themselves is the disastrous economic impact that global warming prevention measures would have. Wake up people. Anyone with a computer, a little time, and some common sense can find many many reasons why this theory is not even close to credible. Don\'t just read articles that support your present opinions, read everything you can find. There is no in-depth analysis to make, really. There is simply too many alternate possibilities and counter-evidence for the theor

In [ ]:
len(neg_reviews)

12500

Ahora, debemos hacer lo mismo con las negativas de test y luego con las positivas.

En nuestro caso, vamos a hacer nuestro propio train/test split, por lo que las a reviews que están en el directorio de test las guardaremos en la misma lista que recién.

Agregar a la lista "neg_reviews" las reviews negativas de test:

In [ ]:
dir_neg_test = "/content/aclImdb/test/neg/"
for f in os.listdir(dir_neg_test):
  with open(f"{dir_neg_test}/{f}") as neg:
    neg_reviews.append(neg.read())

Imprimir el largo de la nueva lista para corroborar que se hayan agregado todas las reviews (deberían tener 25mil)

In [ ]:
len(neg_reviews)

25000

Ahora, hacer lo mismo con las positivas:

In [ ]:
dir_pos_train = "/content/aclImdb/train/pos/"
dir_pos_test = "/content/aclImdb/test/pos/"
pos_reviews = []

for f in os.listdir(dir_pos_train):
  with open(f"{dir_pos_train}/{f}") as pos:
    pos_reviews.append(pos.read())

for f in os.listdir(dir_pos_test):
  with open(f"{dir_pos_test}/{f}") as pos:
    pos_reviews.append(pos.read())

print(pos_reviews[:3])
print(len(pos_reviews))

["Josef Von Sternberg directs this magnificent silent film about silent Hollywood and the former Imperial General to the Czar of Russia who has found himself there. Emil Jannings won a well-deserved Oscar, in part, for his role as the general who ironically is cast in a bit part in a silent picture as a Russian general. The movie flashes back to his days in Russia leading up to the country's fall to revolutionaries. William Powell makes his big screen debut as the Hollywood director who casts Jannings in his film. The film serves as an interesting look at the fall of Russia and at an imitation of behind-the-scenes Tinseltown in the early days. Von Sternberg delivers yet another classic, and one that is filled with the great elements of romance, intrigue, and tragedy.", 'This is such a great movie "Call Me Anna" because it shows how a person has suffered for so long without knowing what was wrong with her. For Patty Duke to come out in the publics eye and tell her story is an inspiratio

Deberían tener 25mil reviews de cada tipo.

Ahora, almacenaremos estos datos en un dataframe de pandas para trabajar de forma más simple.

In [ ]:
pos_df = pd.DataFrame(pos_reviews, columns=["REVIEW"])
pos_df["TARGET"] = "POS"
neg_df = pd.DataFrame(neg_reviews, columns=["REVIEW"])
neg_df["TARGET"] = "NEG"

df = pd.concat([pos_df, neg_df], axis="rows")
df.head()

,REVIEW,TARGET
0,Josef Von Sternberg directs this magnificent s...,POS
1,"This is such a great movie ""Call Me Anna"" beca...",POS
2,I'd waited for some years before this movie fi...,POS
3,Allow me to just get to the bottom line here: ...,POS
4,An excellent movie and great example of how sc...,POS


In [ ]:
df.shape

(50000, 2)

In [ ]:
df.sample(5)

,REVIEW,TARGET
14435,"First of all, I would like to say that I am a ...",NEG
21955,"What can i say about Tromeo and Juliet, other ...",POS
21546,Very good film. Very good documentary.<br /><b...,POS
5316,Earlier today I got into an argument on why so...,NEG
20318,I saw this movie on a fluke.I was standing on ...,POS


Ya tenemos nuestro dataframe listo para trabajar.

### Spacy

Como dijimos anteriormente, trabajaremos con la librería spacy. Siempre debemos importarla e instanciarla llamando a nuestro lenguaje. Si queremos instanciarla con un lenguaje que no tenemos descargado, nos dará un error y ahi podemos copiar y pegar el código para descargar el mismo.

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

Para tokenizar un texto en spacy, simplemente utilizamos el objeto que instanciamos (que en este caso llamamos nlp)

Por ejemplo:

In [ ]:
nlp("Hola como estás")

Hola como estás

Si queremos acceder a cada uno de los tokens, podemos utilizar por ejemplo un bucle for:

In [ ]:
for token in nlp("Hola como estás"):
  print(token)
  print("---")

Hola
---
como
---
estás
---


### Stop words

En spacy, tenemos para cada idioma un listado de stop words por defecto (que podemos modificar agregando o quitando las que necesitemos).

Ejecutando la siguiente celda, podemos ver el listado que viene por defecto para el idioma inglés.

In [ ]:
nlp.Defaults.stop_words

{"'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'four',
 'from',
 'fron

Si queremos agregar una stopword, podemos hacerlo con el método .add() de las listas.

Por ejemplo, imaginen que queremos agregar la palabra "test".

Primero validamos si existe en la lista:


In [ ]:
"test" in nlp.Defaults.stop_words

True

No existe, la agreguemos:

In [ ]:
nlp.Defaults.stop_words.add("test")

In [ ]:
"test" in nlp.Defaults.stop_words

True

Ahora si existe.

Para saber si un token es una stopword o no, podemos utilizar el atributo is_stop de un token.

Veamos un ejemplo:

In [ ]:
for token in nlp("My name is Federico. I am 24 years old and I live in Córdoba."):
  if token.is_stop:
    print(f"La palabra: {token.text} es una stop word.")

La palabra: My es una stop word.
La palabra: name es una stop word.
La palabra: is es una stop word.
La palabra: I es una stop word.
La palabra: am es una stop word.
La palabra: and es una stop word.
La palabra: I es una stop word.
La palabra: in es una stop word.


De esta forma, podemos limpiar las stop words de un texto. Veamos un ejemplo en el que limpiamos las stop words del texto "My name is Federico. I am 24 years old and I live in Córdoba.".

In [ ]:
def clean_stop_words(text):
  clean_text = []
  for token in nlp(text):
    if not token.is_stop:
      clean_text.append(token.text)

  return " ".join(clean_text)

In [ ]:
texto = "My name is Federico. I am 24 years old and I live in Córdoba."

clean_stop_words(texto)

'Federico . 24 years old live Córdoba .'

Vemos que nos limpio las stop words, pero además necesitaríamos pasar el texto a minúsculas y eliminar los signos de puntuación.

Para lo primero, podemos utilizar la función lower() de python.

Para lo segundo, los tokens tienen el atributo token.is_punct.

EJERCICIO: Crear una nueva función (basada en la que definimos recien) que se llame clean_text y además de eliminar stop words, elimine signos de puntuación y convierta todo a minúsculas.

In [ ]:
def clean_text(text):
  clean_text = []
  for token in nlp(text):
    if not token.is_stop and not token.is_punct:
      clean_text.append(token.text.lower())

  return " ".join(clean_text)

Probamos la función con el mismo texto que recién:

In [ ]:
texto = "My name is Federico. I am 24 years old and I live in Córdoba."

clean_text(texto)

'federico 24 years old live córdoba'

Ahora, si quisiéramos aplicar esta función a nuestro dataset entero, como lo haríamos???

### Raiz (lemma)

En spacy, también podemos llevar a las palabras a su raiz de una forma muy simple utilizando el atributo .lemma_ (recuerden que finaliza con _) de los tokens.

Veamos un ejemplo:

In [ ]:
text = """Reeves look like an Oscar winner this film bites (pun not intended). 
The best thing about it is the box of eRATicate in the 2nd segment"""

for token in nlp(text):
    print(f"ORIGINAL {token.text}, LEMMA: {token.lemma_}")

ORIGINAL Reeves, LEMMA: reeve
ORIGINAL look, LEMMA: look
ORIGINAL like, LEMMA: like
ORIGINAL an, LEMMA: an
ORIGINAL Oscar, LEMMA: Oscar
ORIGINAL winner, LEMMA: winner
ORIGINAL this, LEMMA: this
ORIGINAL film, LEMMA: film
ORIGINAL bites, LEMMA: bite
ORIGINAL (, LEMMA: (
ORIGINAL pun, LEMMA: pun
ORIGINAL not, LEMMA: not
ORIGINAL intended, LEMMA: intend
ORIGINAL ), LEMMA: )
ORIGINAL ., LEMMA: .
ORIGINAL 
, LEMMA: 

ORIGINAL The, LEMMA: the
ORIGINAL best, LEMMA: good
ORIGINAL thing, LEMMA: thing
ORIGINAL about, LEMMA: about
ORIGINAL it, LEMMA: -PRON-
ORIGINAL is, LEMMA: be
ORIGINAL the, LEMMA: the
ORIGINAL box, LEMMA: box
ORIGINAL of, LEMMA: of
ORIGINAL eRATicate, LEMMA: eRATicate
ORIGINAL in, LEMMA: in
ORIGINAL the, LEMMA: the
ORIGINAL 2nd, LEMMA: 2nd
ORIGINAL segment, LEMMA: segment


EJERCICIO: A la función clean text, agregarle que convierta el texto a lemma.

In [ ]:
def clean_text(text):
  clean_text = []
  for token in nlp(text):
    if not token.is_stop and not token.is_punct:
      clean_text.append(token.lemma_.lower())

  return " ".join(clean_text)

In [ ]:
text = """Reeves look like an Oscar winner this film bites (pun not intended). 
The best thing about it is the box of eRATicate in the 2nd segment"""
clean_text(text)

'reeve look like oscar winner film bite pun intend \n good thing box eraticate 2nd segment'

Vemos que todavía quedan caracteres especiales como por ejemplo \n. Ya veremos en próximas clases como limpiar este tipo de elementos utilizando expresiones regulares. 

Por ahora, podemos utilizar la función .replace() de los strings. Por ejemplo:

In [ ]:
"hola como estas".replace("como", "")

'hola  estas'

### Bag of words

Para vectorizar con bag of words, utilizaremos sklearn. 

En sklearn, este elemento se llama CountVectorizer.

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html


Sigue la lógica de fit/transform.

Podemos ver algunos parámetros importantes como por ejemplo:

- ngram_range
- lowercase
- stop_words
- strip_accents


Antes de aplicar count vectorizer sobre nuestro df, vamos a aplicarle nuestra función "clean_text".

EJERCICIO: Aplicar clean_text a todo nuestro dataframe. 

Este proceso puede tomar más de media hs para el dataset que tenemos, por lo tanto, nos quedaremos únicamente con las primeras 5mil filas para poder ejecutar el código en clases. Luego ustedes pueden probarlo con el dataset completo.

In [ ]:
# COMENTAR ESTA CELDA SI QUIEREN TRABAJAR CON EL DATASET COMPLETO (les puede tomar 30 min el preprocesamiento)
pos_samples = df[df.TARGET=='POS'].head(2500)
neg_samples = df[df.TARGET=='NEG'].head(2500)

df = pd.concat([pos_samples, neg_samples])

In [ ]:
%%time
df["REVIEW"] = df["REVIEW"].apply(clean_text)

CPU times: user 4min 1s, sys: 2.32 s, total: 4min 4s
Wall time: 4min 3s


In [ ]:
df.head()

,REVIEW,TARGET
0,josef von sternberg direct magnificent silent ...,POS
1,great movie anna show person suffer long know ...,POS
2,wait year movie finally get release england wa...,POS
3,allow line get 3 kid age 5 10 consider trip th...,POS
4,excellent movie great example scary movie show...,POS


Ahora, debemos hacer train_test_split. Utilizar como random_state 0 y test_size de 0.2

In [ ]:
from sklearn.model_selection import train_test_split

X = df.REVIEW.copy()
y = df.TARGET.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Ahora si, importemos count vectorizer y lo apliquemos en nuestro texto.

Recuerden: fit solo sobre train.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer()

cv.fit(X_train)

X_train = cv.transform(X_train)
X_test = cv.transform(X_test)

In [ ]:
X_train

<4000x29434 sparse matrix of type '<class 'numpy.int64'>'
	with 342225 stored elements in Compressed Sparse Row format>

In [ ]:
X_test

<1000x29434 sparse matrix of type '<class 'numpy.int64'>'
	with 81583 stored elements in Compressed Sparse Row format>

Ahora, además podemos ver las "features" con la siguiente función de nuestro count vectorizer (en la siguiente celda, cambien el nombre del vectorizer que ustedes hayan utilizado).

In [ ]:
cv.get_feature_names()

['00',
 '000',
 '00001',
 '006',
 '007',
 '0079',
 '0080',
 '0083',
 '00pm',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '0ne',
 '10',
 '100',
 '1000',
 '1001',
 '100m',
 '100th',
 '101',
 '102',
 '102nd',
 '103',
 '104',
 '1040s',
 '105',
 '105lbs',
 '106',
 '108',
 '109',
 '10pm',
 '10star',
 '10th',
 '11',
 '110',
 '1100',
 '11001001',
 '112',
 '115',
 '116',
 '11th',
 '12',
 '120',
 '12383499143743701',
 '125',
 '12th',
 '13',
 '1300',
 '131',
 '132',
 '134',
 '135',
 '137',
 '13k',
 '13th',
 '14',
 '140',
 '146',
 '149',
 '1492',
 '14a',
 '14th',
 '15',
 '150',
 '1500',
 '157',
 '15mins',
 '15minutes',
 '16',
 '160lbs',
 '163',
 '16mm',
 '16th',
 '16ème',
 '17',
 '1775',
 '1798',
 '17th',
 '18',
 '1800',
 '1812',
 '1820',
 '1824',
 '1830',
 '1836',
 '1837',
 '1838',
 '1840',
 '1846',
 '1847',
 '1850',
 '1853',
 '1854',
 '1855',
 '1860',
 '1861',
 '1865',
 '1873',
 '1876',
 '188',
 '1880',
 '1888',
 '1890',
 '1890s',
 '1895',
 '1896',
 '1898',
 '18a',
 '18th',
 '19',
 

Vamos a encontrar muchisimas palabras que no tienen sentido y no aportan nada a nuestro modelo, o caracteres como "__________________________________________________________________".
Todo esto podríamos tenerlo en cuenta para la etapa de preprocesamiento.

Con el X_train y X_test que generamos con nuestro countVectorizer ya podríamos entrenar un modelo.

Ahora, aplicaremos TF IDF


### TFIDF

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

Generar X_train, X_test, y_train e y_test de nuevo, ya que las modificamos anteriormente con el count vectorizer:

In [ ]:
X = df.REVIEW.copy()
y = df.TARGET.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Ahora, importar tfidf vectorizer y aplicarlo sobre X_train y X_test

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
X_train

<4000x29434 sparse matrix of type '<class 'numpy.float64'>'
	with 342225 stored elements in Compressed Sparse Row format>

In [ ]:
X_test

<1000x29434 sparse matrix of type '<class 'numpy.float64'>'
	with 81583 stored elements in Compressed Sparse Row format>

Entrenar un SVC con los datos ya vectorizados.

Utilizaremos: 
- random_state=0
- C=0.5

In [ ]:
from sklearn.svm import SVC
clf = SVC(random_state=0, C=0.5)
clf.fit(X_train, y_train)

SVC(C=0.5, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

Ahora, como siempre hicimos, podemos medir métricas. Por ejemplo, imprimir el classification report.

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_train, clf.predict(X_train)))
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

         NEG       0.98      0.96      0.97      1981
         POS       0.96      0.99      0.97      2019

    accuracy                           0.97      4000
   macro avg       0.97      0.97      0.97      4000
weighted avg       0.97      0.97      0.97      4000

              precision    recall  f1-score   support

         NEG       0.90      0.75      0.82       519
         POS       0.77      0.91      0.84       481

    accuracy                           0.83      1000
   macro avg       0.84      0.83      0.83      1000
weighted avg       0.84      0.83      0.83      1000

